In [1]:
import os
import time
import json
import numpy as np
import openslide
from histoplus.helpers.tissue_detection import detect_tissue_on_wsi
from histoplus.helpers.segmentor import CellViTSegmentor
from histoplus.extract import extract
from draw_utils import *
from openslide.deepzoom import DeepZoomGenerator

/home/sjhong/miniconda3/envs/cellvit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wsi_path = '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-028_08.svs'
# wsi_path = "/home/sjhong/segmentation/1-028_08_002.svs"

In [ ]:
"""x0, y0 = (20450,43558)
x1, y1 = (31436, 56318)

# x0, y0 = (0,0)
# x1, y1 = (10986, 12760)
w = x1-x0
h= y1-y0

slide = openslide.OpenSlide(wsi_path)
dz = DeepZoomGenerator(slide, tile_size=224, overlap=0, limit_bounds=False)

tiles = tiles_covering_base_rect(slide, dz, 16, x0, y0, w, h,
                                tile_size=224, overlap=0, return_spans=False)
len(tiles)"""

754

In [ ]:
# 현재 결과 세팅
tissue_coords, dz_level = detect_tissue_on_wsi(openslide.OpenSlide(wsi_path),
   										 matter_threshold = 0.5,)
#0.5 22639
#0.4 22138
#0.3 21228
'''tiles_arr = np.array(tiles, dtype=int)
tissue_arr = np.array(tissue_coords, dtype=int).copy()

# (1) view를 이용한 빠른 교집합 (메모리 복사 없음)
tiles_view = tiles_arr.view([('', tiles_arr.dtype)] * tiles_arr.shape[1])
tissue_view = tissue_arr.view([('', tissue_arr.dtype)] * tissue_arr.shape[1])

inter = np.intersect1d(tiles_view, tissue_view)

# (2) 다시 (tx, ty) 튜플 형태로 복원
roi_coords = inter.view(tiles_arr.dtype).reshape(-1, 2)
print(f"교집합 개수: {len(roi_coords)}")'''

segmentor = CellViTSegmentor.from_histoplus( mpp=0.25,
										mixed_precision=True, # GPU AMP 사용 권장 
										inference_image_size=588, )

start = time.time()

results = extract(
    slide=openslide.OpenSlide(wsi_path),
    coords=tissue_coords, #roi_coords,
    deepzoom_level=dz_level,
    segmentor=segmentor,
    batch_size=48,  # GPU 여유 있으면 20~24까지
)

end = time.time()
elapsed = end - start

# 초 → 분, 초로 변환
minutes, seconds = divmod(elapsed, 60)
print(f"processed:  {int(minutes):2d} m  {seconds:4.1f} s")

2025-10-24 14:15:03.319 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-24 14:15:03.321 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-24 14:15:16.771 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 22639 tiles with Otsu's threshold.
2025-10-24 14:15:20.410 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.


In [ ]:
INFERENCE_IMAGE_SIZE = 588
target_mpp = 8.0 #4.0
thres =0.5# 0.18
tile_size = 224
infer_overlap = 0
overall = 0.6 # 0.4
gau_ker = 7
min_size = 10
sobel_ker = 21
minsize = 100
mor = 'O3+C5del2'#original O3 only

In [ ]:
output_dir = './output/1_028_08_post_tuning/'
os.makedirs(output_dir, exist_ok = True)
json_path = f"{output_dir}/whole_{wsi_path.split('/')[-1][:-4]}_{thres}_{overall}_{gau_ker}_{minsize}_{mor}.json"
results.save(json_path)
print(f"OK → {json_path}")

OK → ./output/1_028_08_post_tuning//seg_1-028_08_0.5_0.6_7_100_O3+C5del2.json


In [7]:
#json_path = f"{output_dir}/seg_{wsi_path.split('/')[-1][:-4]}_{INFERENCE_IMAGE_SIZE}_{target_mpp}_{thres}_{tile_size}_{infer_overlap}.json"
#OK → ./output/1_028_08//seg_1-028_08_588_4.0_0.18_224_128.json

# === 
# output_dir = './output/1_028_08_patch/'

In [8]:
try:
    del results
except:
    pass
# wsi_path = '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-028_08.svs'
# json_path = './output/1_028_08//seg_1-028_08_588_8.0_0.2_224_128.json'
# JSON 로딩(여러 파일이면 순차 로딩 + 레코드 합치기)
with (open(json_path, "r", encoding="utf-8") if not json_path.endswith(".gz") else __import__("gzip").open(json_path, "rt", encoding="utf-8")) as f:
    data = json.load(f, parse_float=float)

In [9]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
tile_size  = 224         # JSON 생성 시 사용한 DeepZoom tile_size와 동일
overlap    = 0           # JSON 생성 시 overlap과 동일
# level      = 13          # JSON 항목의 level(당신 로그에선 16)
level      = 16          # JSON 항목의 level(당신 로그에선 16)
limit_bounds = False     # JSON 생성 시 설정과 동일
# (선택) ROI (base 좌표)
x0, y0 = (20450, 43558)
x1, y1 = (31436, 56318)

def get_bounds_offset(slide):
    bx = int(float(slide.properties.get('openslide.bounds-x', 0)))
    by = int(float(slide.properties.get('openslide.bounds-y', 0)))
    bw = int(float(slide.properties.get('openslide.bounds-width', 0)))
    bh = int(float(slide.properties.get('openslide.bounds-height', 0)))
    return bx, by, bw, bh

def tiles_covering_base_rect(slide, dz, level, x0, y0, w, h,
                             tile_size=224, overlap=0,
                             return_spans=False):
    """
    base(level-0) 사각형(x0,y0,w,h)을 덮는 DZ(level) 타일 범위 빠르게 계산.
    - return_spans=False: [(level, tx, ty), ...] 리스트
    - return_spans=True : [(ty, tx0, tx1), ...] 행 단위 span (더 빠른 순회에 유리)
    """
    # 1) DZ level 해상도 대비 base 배율 + bounds 오프셋
    W0, H0 = slide.dimensions
    Wd, Hd = dz.level_dimensions[level]
    sx, sy = W0 / float(Wd), H0 / float(Hd)  # dz픽셀→base픽셀 배율
    bx, by, _ , _  = get_bounds_offset(slide)

    # 2) base rect → DZ 픽셀 좌표 (오프셋 제거, 배율 반영)
    #    우측/하단 경계는 포함되도록 -eps(=1e-6) 대신 -1 픽셀 후 나눔
    dx0 = (x0 - bx) / sx
    dy0 = (y0 - by) / sy
    dx1 = (x0 + w - 1 - bx) / sx
    dy1 = (y0 + h - 1 - by) / sy

    # 3) DZ 타일 인덱스 범위 (step = tile_size - overlap)
    step = tile_size - overlap
    tx0 = int(math.floor(dx0 / step))
    ty0 = int(math.floor(dy0 / step))
    tx1 = int(math.floor(dx1 / step))
    ty1 = int(math.floor(dy1 / step))

    # 4) 그리드로 클램프
    tiles_w, tiles_h = dz.level_tiles[level]
    tx0 = max(0, min(tx0, tiles_w - 1))
    tx1 = max(0, min(tx1, tiles_w - 1))
    ty0 = max(0, min(ty0, tiles_h - 1))
    ty1 = max(0, min(ty1, tiles_h - 1))

    if return_spans:
        # 행 단위 span으로 반환: (ty, tx0, tx1)
        return [(ty, tx0, tx1) for ty in range(ty0, ty1 + 1)]
    else:
        # 개별 타일 인덱스 나열
        return [(tx, ty)
                for ty in range(ty0, ty1 + 1)
                for tx in range(tx0, tx1 + 1)]
    return {(tx, ty) for ty in range(ty0, ty1+1) for tx in range(tx0, tx1+1)}

def tile_rect_base(slide, dz, level, tx, ty, tile_size, overlap):
    # DeepZoom level 좌표 → base(level-0) 좌표 박스
    W0, H0 = slide.dimensions
    Wd, Hd = dz.level_dimensions[level]
    sx, sy = W0 / float(Wd), H0 / float(Hd)
    bx, by, _, _ = get_bounds_offset(slide)  # limit_bounds=True면 꼭 동일하게!

    step = tile_size - overlap
    dx0 = tx * step
    dy0 = ty * step
    dx1 = min(dx0 + tile_size, Wd)  # 레벨 이미지 경계 클램프
    dy1 = min(dy0 + tile_size, Hd)

    # base 좌표로 매핑(정수 반올림)
    x0 = int(round(dx0 * sx + bx))
    y0 = int(round(dy0 * sy + by))
    x1 = int(round(dx1 * sx + bx))
    y1 = int(round(dy1 * sy + by))
    return x0, y0, x1, y1  # [x0,x1), [y0,y1)

def rects_intersect(a, b):
    ax0, ay0, ax1, ay1 = a
    bx0, by0, bx1, by1 = b
    return (ax0 < bx1) and (bx0 < ax1) and (ay0 < by1) and (by0 < ay1)


In [10]:

items = data.get("masks", data.get("cell_masks", []))  # 두 키 중 하나
items_lvl = [t for t in items if int(t.get("level", -1)) == level]

# 타일 인덱스를 base 좌표 사각형으로 변환
rects = [tile_rect_base(slide, dz, level, int(t["x"]), int(t["y"]), tile_size, overlap)
         for t in items_lvl]
len(rects)


754

In [11]:
# 썸네일
W0, H0 = slide.dimensions
max_edge = 2500
scale = min(max_edge / W0, max_edge / H0)
thumb = slide.get_thumbnail((int(W0*scale), int(H0*scale)))

In [12]:
'''fig, ax = plt.subplots(figsize=(6, 6*H0/W0))
ax.imshow(thumb)
ax.set_title(f"JSON tiles on WSI • level={level}, tile_size={tile_size}, overlap={overlap} • N={len(rects)}")
ax.axis('off')

# 타일 사각형(성능 위해 필요 시 간격 샘플링)
step_plot = max(1, len(rects)//20000)  # 2만개 이상이면 간격 추려 그림
for (bx0, by0, bx1, by1) in rects[::step_plot]:
    rx, ry = bx0*scale, by0*scale
    rw, rh = (bx1-bx0)*scale, (by1-by0)*scale
    ax.add_patch(Rectangle((rx, ry), rw, rh, fill=False, lw=0.25, ec='b', alpha=0.5))

# ROI 박스(선택)
ax.add_patch(Rectangle((x0*scale, y0*scale), (x1-x0)*scale, (y1-y0)*scale,
                       fill=False, lw=2.0, ec='y'))
plt.show()'''

'fig, ax = plt.subplots(figsize=(6, 6*H0/W0))\nax.imshow(thumb)\nax.set_title(f"JSON tiles on WSI • level={level}, tile_size={tile_size}, overlap={overlap} • N={len(rects)}")\nax.axis(\'off\')\n\n# 타일 사각형(성능 위해 필요 시 간격 샘플링)\nstep_plot = max(1, len(rects)//20000)  # 2만개 이상이면 간격 추려 그림\nfor (bx0, by0, bx1, by1) in rects[::step_plot]:\n    rx, ry = bx0*scale, by0*scale\n    rw, rh = (bx1-bx0)*scale, (by1-by0)*scale\n    ax.add_patch(Rectangle((rx, ry), rw, rh, fill=False, lw=0.25, ec=\'b\', alpha=0.5))\n\n# ROI 박스(선택)\nax.add_patch(Rectangle((x0*scale, y0*scale), (x1-x0)*scale, (y1-y0)*scale,\n                       fill=False, lw=2.0, ec=\'y\'))\nplt.show()'

In [13]:
def _unique_xml_path(path: str) -> str:
    if not path.lower().endswith(".xml"):
        path = path + ".xml"
    if not os.path.exists(path):
        return path
    base, ext = os.path.splitext(path)
    k = 1
    while True:
        cand = f"{base}_{k}{ext}"
        if not os.path.exists(cand):
            return cand
        k += 1
        
def test_annotation(json_path, tile_size, overlap):
    wsi_path = '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-028_08.svs'
    # wsi_path = "/home/sjhong/segmentation/1-028_08_002.svs"
    wsi_name = '1-028_08'
    x0, y0 = (20450,43558)
    x1, y1 = (31436, 56318)
    
    # x0, y0 = (0,0)
    # x1, y1 = (10986, 12760)
    w = x1-x0
    h= y1-y0
    
    # ── 사용 예 ───────────────────────────────────────────────────
    slide = openslide.OpenSlide(wsi_path)
    dz = DeepZoomGenerator(slide, tile_size=tile_size, overlap=overlap, limit_bounds=False)
    # lvl = 13
    lvl = 16
    
    # x0, y0 = 72931, 28727 # 좌측 상단 좌표
    # w, h   = 8465, 2604

    # (A) 낱개 타일 리스트
    tiles = tiles_covering_base_rect(slide, dz, lvl, x0, y0, w, h,
                                    tile_size=tile_size, overlap=overlap, return_spans=False)
    if len(tiles) == 1:
        print("I founded this tile:",tiles[0])
        tile = tiles[0]
    else:
        print(f"tile count: {len(tiles)}")
        print("first 10:", tiles[:10])  # (level, tx, ty)
        
    found_tiles = [
        t for t in data.get("cell_masks", [])
        if (int(t.get("x", -1)), int(t.get("y", -1))) in tiles
    ]
    if found_tiles:
        print(found_tiles[0])
        print(f"총 {len(found_tiles)}개 tile dict 매칭")

        # cell_type = ['Cancer cell', 'Fibroblasts', 'Macrophages', 'Lymphocytes', 'Apoptotic Body', 'Neutrophils']
        roi_comment = f"ROI: x0={x0}, y0={y0}, width={w}, height={h}"
        # cell_type = ['a']
        annot_path = f"./annotation/histo_{wsi_name}_roi_tuning_test.xml"
        final_path = _unique_xml_path(annot_path)
        
        tiles_to_asap_xml(
            wsi_path=wsi_path,
            tiles=found_tiles,                   # 3k 타일 dict 리스트
            xml_out_path=final_path,
            color_map=COLOR_MAP,
            roi_comment = roi_comment,
            contained = list(COLOR_MAP.keys()),
            max_workers=45,                 # 코어에 맞게 조정
        )
    else:
        print("NO tiles founded.")

In [14]:
test_annotation(json_path, tile_size, overlap)

tile count: 754
first 10: [(45, 97), (46, 97), (47, 97), (48, 97), (49, 97), (50, 97), (51, 97), (52, 97), (53, 97), (54, 97)]
{'level': 16.0, 'x': 45.0, 'y': 97.0, 'width': 224.0, 'height': 224.0, 'masks': [{'cell_id': 0.0, 'cell_type': 'Fibroblasts', 'confidence': 0.9999999973474801, 'coordinates': [[7.0, 67.0], [6.5, 67.5], [6.5, 69.0], [6.0, 69.5], [6.0, 70.5], [5.0, 71.5], [5.0, 74.5], [5.5, 75.0], [5.5, 76.0], [7.0, 77.5], [7.5, 77.5], [8.0, 78.0], [8.5, 78.0], [9.0, 78.5], [11.0, 78.5], [12.5, 77.0], [12.5, 76.0], [13.0, 75.5], [13.0, 75.0], [13.5, 74.5], [13.5, 74.0], [14.0, 73.5], [14.0, 73.0], [14.5, 72.5], [14.5, 69.5], [14.0, 69.0], [14.0, 68.5], [13.0, 67.5], [12.5, 67.5], [12.0, 67.0]], 'centroid': [9.5, 72.5]}, {'cell_id': 1.0, 'cell_type': 'Fibroblasts', 'confidence': 0.9999999927536233, 'coordinates': [[3.5, 97.0], [3.0, 97.5], [2.5, 97.5], [2.5, 98.5], [4.0, 100.0], [4.5, 100.0], [5.0, 100.5], [5.5, 100.5], [6.0, 101.0], [6.5, 101.0], [7.0, 101.5], [8.0, 101.5], [8.5,

./annotation/histo_1-028_08_roi_tuning_test.xml ==
/home/sjhong/segmentation/histoplus/output/1_028_08_post_tuning/seg_1-028_08_0.5_0.5_7.json

./annotation/histo_1-028_08_roi_tuning_test_3.xml ==
/home/sjhong/segmentation/histoplus/output/1_028_08_post_tuning//seg_1-028_08_0.5_0.6_7_150.json


